In [95]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import  matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

from random import sample
seed_list = list(range(10000))

import warnings
warnings.simplefilter('ignore')
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

TODAY = str(datetime.now().year)+str(datetime.now().month)+str(datetime.now().day)

In [96]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\data')

train = pd.read_pickle('fixed2_train.pkl')
test = pd.read_pickle("fixed2_test.pkl")
input_var = pd.read_csv('fixed2_input_var.csv')

sub = pd.read_csv('submission_sample.csv')

os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
experiment_db = pd.read_csv('experiment_DB.csv')

In [97]:
added = []
input_var = list(input_var.columns)
target=['ride_18_20']

In [82]:
all = pd.concat([train,test],axis=0).reset_index(drop=True)

temp2 = all.groupby('bus_route_id')['station_code'].nunique().to_dict()
all['numofstation_byroute'] = all['bus_route_id'].map(temp2)

for col in ['ride_6_12','takeoff_6_12']:

    temp = all.groupby(['date','bus_route_id'])[col].sum().reset_index()
    temp = temp.groupby('bus_route_id')[col].mean().reset_index().rename(columns = {col : 'daily_mean_'+col+'_byroute'})
    
    all = pd.merge(all,temp,how='left',on='bus_route_id').reset_index(drop=True)
    all[col+'proportion_byroute'] = all['daily_mean_'+col+'_byroute']/all['numofstation_byroute']
    
    train = all.loc[:(train.shape[0]-1),]
    test = all.loc[train.shape[0]:,].drop('ride_18_20',axis=1).reset_index(drop=True)
    

added += [a for a in train.columns if 'byroute' in a]

del all

In [84]:
input_var = input_var + added

In [86]:
X_train=train[input_var]
y_train=train[target]

X_test=test[input_var]

In [87]:
NFOLDS = 6
random_seed = sample(seed_list,1)

tscv = TimeSeriesSplit(max_train_size=None, n_splits=NFOLDS)
stk = StratifiedKFold(n_splits=NFOLDS,random_state = random_seed[0],shuffle=True)

In [88]:
experiment_info = 'numstation_byroute, dailymean_byroute, proportion_byroute feature added'

In [89]:
def mean_encoding(col,tr,vl,tst):

    temp = tr.groupby([col])['ride_18_20'].agg(['mean','max','min','count']).reset_index().\
    rename(columns = {'mean' : col+'_'+'mean',
                     'std' : col+'_'+'std',
                     'max' : col+'_'+'max',
                     'min' : col+'_'+'min',
                     'count' : col+'_'+'count'})
    tr_ = pd.merge(tr,temp,how='left',on= col)
    vl_ = pd.merge(vl,temp,how='left',on= col)
    tst_ = pd.merge(tst,temp,how='left',on= col)

    return tr_, vl_,tst_

In [90]:
start = datetime.now()

time = str(start.hour)+'hr'
minute = str(start.minute)+'min'

cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(test.shape[0])
fold_scores = []


for fold_, (tr_index, vl_index) in enumerate(stk.split(X_train,y_train)):
# for fold_, (tr_index, vl_index) in enumerate(tscv.split(train)):    
    print('Fold:', fold_+1)
    tst = X_test[input_var]
    tr, vl = X_train.iloc[tr_index], X_train.iloc[vl_index]
    y_tr, y_vl = y_train.iloc[tr_index], y_train.iloc[vl_index]
    
#     print(tr_index,vl_index)
    
    rf = RandomForestRegressor(random_state=random_seed[0],n_estimators=100,criterion='mse')
    rf.fit(tr,y_tr)
    
    pred = rf.predict(vl)
    
    print(np.sqrt(mean_squared_error(y_vl,pred)))
    cv_train[vl_index] += pred
    cv_pred += rf.predict(tst)
    
    print('-'*40+'\n\n')
    
cv_pred /= NFOLDS

vl_error = np.sqrt(mean_squared_error(np.array(y_train).flatten(),cv_train))

print('cv score:')
print(vl_error)
# print(fold_scores)
# print(best_trees,np.mean(best_trees))

sub['18~20_ride'] = cv_pred

end = datetime.now()

print(end - start)

Fold: 1
2.436357513598846
----------------------------------------


Fold: 2
2.465803317295789
----------------------------------------


Fold: 3
2.4102595946507774
----------------------------------------


Fold: 4
2.412664839153764
----------------------------------------


Fold: 5
2.3948504811152405
----------------------------------------


Fold: 6
2.407399128502118
----------------------------------------


cv score:
2.4213412476887215
1:15:27.545615


In [91]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\submission')
sub['18~20_ride'] = cv_pred
sub.to_csv(TODAY+'_'+time+'_'+minute+'_'+'sub'+'.csv',index=False)

In [92]:
experiment_result = pd.DataFrame({
    'date' : [TODAY],
    'time' : [time],
    'score' : [vl_error],
    'feature_list' : [input_var],
    'model' : 'rf',
    'hyper_param' : 'n_estimatior : 100',
    'explanation' : experiment_info
})

In [93]:
os.chdir('C:\\Users\\yeonjun.in\\Desktop\\연준\\캐글\\제주도\\code\\experiment')
experiment_db = pd.concat([experiment_db,experiment_result],axis=0)
experiment_db.to_csv('experiment_DB.csv',index=False)

In [94]:
experiment_db

,date,time,score,feature_list,model,hyper_param,explanation
0,20191117,first,2.960000,none,NaN,NaN,NaN
1,20191117,22hr,2.838222,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,NaN,NaN
2,20191117,23hr,2.836819,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,day_type 추가
3,20191117,23hr,2.673055,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,"station_code, station_name, bus_route_id freq ..."
4,20191118,1hr,2.663316,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,public data usage
5,20191118,14hr,2.634972,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,public data usage
6,20191118,16hr,2.419647,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,"station_code,name,bus_route_id group by ride_6..."
7,20191118,16hr,2.419647,"['in_out', 'latitude', 'longitude', 'ride_6_7'...",rf,n_estimatior : 10,"station_code,name,bus_route_id group by ride_6..."
0,20191120,15hr,2.421341,"[in_out, latitude, longitude, ride_6_7, ride_7...",rf,n_estimatior : 100,"numstation_byroute, dailymean_byroute, proport..."
